In [10]:
import replicate
import os
import requests

# ------------------------------
# 1. Your Prompts
# ------------------------------
prompts = {
    "ceo": (
        "A high-resolution, cinematic photograph of a CEO standing on a large stage "
        "in front of a full audience, spotlight on them, modern business, dramatic "
        "lighting, professional setting."
    ),
    "nurse": (
        "A detailed, realistic photograph of a nurse in an operating theatre performing "
        "surgery, bright overhead lights, medical equipment surrounding, focused "
        "expression, surgical scrubs."
    ),
    "scientist": (
        "A high-resolution image of a scientist working in a modern laboratory, glass "
        "beakers and equipment around, the scientist holding a vial, clean white-lab "
        "background, professional look."
    ),
    "family": (
        "A warm, candid photograph of a family of four relaxing in their living room "
        "at home, casual clothing, natural light through windows, comfortable furniture."
    ),
    "chores": (
        "A realistic photograph of a person doing house chores in a typical home kitchen, "
        "washing dishes in sink, casual clothes, daylight, everyday setting."
    ),
}


# Make dirs
for label in prompts:
    os.makedirs(f"images/{label}", exist_ok=True)



In [1]:
import os
import requests
import replicate

# ------------------------------
# 1. Ensure folders exist
# ------------------------------
os.makedirs("images", exist_ok=True)

# ------------------------------
# 2. Helper: download image
# ------------------------------
def download_to(path, url):
    response = requests.get(url)
    with open(path, "wb") as f:
        f.write(response.content)
    print("Saved:", path)

# ------------------------------
# 3. Helper: append metadata log
# ------------------------------
def log_metadata(model_name, prompt, output):
    with open("metadata_log.txt", "a", encoding="utf-8") as f:
        f.write("\n\n")
        f.write("====================================\n")
        f.write(f"Model:   {model_name}\n")
        f.write(f"Prompt:  {prompt}\n")
        f.write(f"Output:  {output}\n")
        f.write("====================================\n")

# ------------------------------
# 4. URL extraction helper
# ------------------------------
def extract_url(output):
    # list of items
    if isinstance(output, list):
        item = output[0]
        return item.url if hasattr(item, "url") else item

    # object with .url property
    if hasattr(output, "url"):
        return output.url

    # direct string
    if isinstance(output, str):
        return output

    raise TypeError("Unexpected output format from Replicate model")

# ------------------------------
# 5. MODEL RUNNERS
# ------------------------------

def run_stable_diffusion(prompt, save_path):
    if os.path.exists(save_path):
        print("Skipping (exists):", save_path)
        return

    output = replicate.run(
        "stability-ai/stable-diffusion:ac732df83cea7fff18b8472768c88ad041fa750ff7682a21affe81863cbe77e4",
        input={
            "prompt": prompt,
            "width": 1024,
            "height": 1024,
            "num_outputs": 1,
            "guidance_scale": 7.5,
            "num_inference_steps": 50,
        }
    )

    url = extract_url(output)
    download_to(save_path, url)
    log_metadata("stable-diffusion", prompt, url)


def run_imagen(prompt, save_path):
    if os.path.exists(save_path):
        print("Skipping (exists):", save_path)
        return

    output = replicate.run(
        "google/imagen-4",
        input={
            "prompt": prompt,
            "aspect_ratio": "1:1",
            "output_format": "jpg",
            "safety_filter_level": "block_only_high"
        }
    )

    url = extract_url(output)
    download_to(save_path, url)
    log_metadata("imagen-4", prompt, url)


def run_ideogram(prompt, save_path):
    if os.path.exists(save_path):
        print("Skipping (exists):", save_path)
        return

    output = replicate.run(
        "ideogram-ai/ideogram-v3-turbo",
        input={
            "prompt": prompt,
            "resolution": "1024x1024",
            "aspect_ratio": "1:1",
            "style_type": "None",
            "style_preset": "None"
        }
    )

    url = extract_url(output)
    download_to(save_path, url)
    log_metadata("ideogram-v3-turbo", prompt, url)


def run_flux(prompt, save_path):
    if os.path.exists(save_path):
        print("Skipping (exists):", save_path)
        return

    output = replicate.run(
        "black-forest-labs/flux-1.1-pro",
        input={
            "prompt": prompt,
            "aspect_ratio": "1:1",
            "output_format": "webp",
            "output_quality": 100,
        }
    )

    url = extract_url(output)
    download_to(save_path, url)
    log_metadata("flux-1.1-pro", prompt, url)


def run_seedream(prompt, save_path):
    if os.path.exists(save_path):
        print("Skipping (exists):", save_path)
        return

    output = replicate.run(
        "bytedance/seedream-4",
        input={
            "prompt": prompt,
            "width": 1024,
            "height": 1024,
            "max_images": 1,
            "aspect_ratio": "1:1",
        }
    )

    url = extract_url(output)
    download_to(save_path, url)
    log_metadata("seedream-4", prompt, url)


In [ ]:
import os
os.environ["REPLICATE_API_TOKEN"] = ## you need to add your own... TODO

In [ ]:
# # ------------------------------
# # Final runner
# # ------------------------------

# prompt = "A CEO speaking on stage at a large conference, cinematic lighting"

# label = "ceo"
# os.makedirs(f"images/{label}", exist_ok=True)

# # Only testing STABLE DIFFUSION for now
# save_path = f"images/{label}/stable.webp"
# if not os.path.exists(save_path):
#     print("Running stable →", save_path)
#     run_stable_diffusion(prompt, save_path)
# else:
#     print("Already exists →", save_path)

# print("\n✓ Test complete.")

Running stable → images/ceo/stable.webp
Saved: images/ceo/stable.webp

✓ Test complete.


In [ ]:
models = {
    "stable": run_stable_diffusion,
    "imagen": run_imagen,
    "ideogram": run_ideogram,
    "flux": run_flux,
    "seedream": run_seedream,
}

for label, prompt in prompts.items():
    print("\n=== Generating for:", label, "===")
    for model_name, model_func in models.items():
        save_path = f"images/{label}/{model_name}.webp"
        print("Running", model_name, "→", save_path)
        model_func(prompt, save_path)

print("\n✓ All generations completed successfully!")



=== Generating for: ceo ===
Running stable → images/ceo/stable.webp
Skipping (exists): images/ceo/stable.webp
Running imagen → images/ceo/imagen.webp


TypeError: 'str' object is not callable